In [ ]:
#testing results with Lasso and Elastic Net - expected 15/260 most important features
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, ElasticNet

from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ConfusionMatrix

In [ ]:
all_data = pd.read_csv('../data/election_census_final')
description = pd.read_csv('../data/var_description.csv')

In [ ]:
y = all_data.target

In [ ]:
X = all_data.iloc[:, 8:268]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit(X_test)
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=0, positive=True)
model = elastic_net.fit(X_train_scaled, y_train)
features = pd.DataFrame(model.coef_, columns=['Importance'], index=range(260))
features.head(2)

In [ ]:
features_names_df = pd.DataFrame(X.iloc[0, :].T)
features_names_df.reset_index(inplace=True)
features_names_df.columns = ['Code','0']
features_names_df = features_names_df.drop(columns=['0'])
features_names_df.index = range(260)
features_names_df.head(2)

In [ ]:
new_df = pd.merge(features, features_names_df, left_index=True, right_index=True)
new_df.head(5)

In [ ]:
description.head()
description.columns=(["Code", "Label"])

In [ ]:
top_features_df = pd.merge(new_df, description, on='Code')
top_features_df.head(5)

In [ ]:
pd.set_option('display.max_colwidth', -1)
top_features_df.sort_values(by='Importance', ascending=False)

In [ ]:
#this is way we could not use Lass0 --> too many correlated features
lasso_reg = Lasso()
lass_features  = lasso_reg.fit(X_train_scaled, y_train).coef_

In [33]:
#check non-linear SVM Classification 
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.svm import LinearSVC

polynomial_svm_clf = Pipeline([
    ('poly_features', PolynomialFeatures(degree=3)),
    ('scaler', StandardScaler()),
    ('svm_clf', LinearSVC(C=10, loss='hinge'))
])
X, y = make_moons()
model = polynomial_svm_clf.fit(X, y)


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [34]:
model.classes_

array([0, 1])

In [40]:
model.score(X, y)

1.0